In [1]:
import json
import gzip
import os

import pandas as pd
import numpy as np
import requests

from config_data import config

data_path = os.path.join("by_jobs")
text_processing_url = config.text_processing_url
n_vacancies_to_save = 10_000

pd.options.display.max_rows = 100
pd.set_option('display.max_columns', None)

### Read data from "by_jobs" folder

In [3]:
def load_data(data_path):
    """
    Load all data to pandas.DataFrame
    
    :param str data_path: Path to folder with data 
    :return pd.DataFrame data:
    """
    docs_info = []
    docs_text = []
    file_names = os.listdir(data_path)

    for file in file_names:
        if "text" in file:
            with gzip.open(os.path.join(data_path, file), "rb") as f:
                for line in f:
                    vacancy = json.loads(line)
                    docs_text.append(vacancy)
        else:
            with gzip.open(os.path.join(data_path, file), "rb") as inf:
                for line in inf:
                    vacancy = json.loads(line)
                    docs_info.append(vacancy)

    assert len(docs_info) == len(docs_text)
    assert "id" in docs_info[0].keys()
    assert "id_job" in docs_text[0].keys()

    docs_info = pd.DataFrame(docs_info)
    docs_text = pd.DataFrame(docs_text)
    docs_info.drop_duplicates(["id"], inplace=True)
    docs_text.drop_duplicates(["id_job"], inplace=True)
    data = docs_info.merge(docs_text, left_on='id', right_on='id_job', how='outer')
    
    return data

In [21]:
full_df = load_data(data_path)
print(full_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90081 entries, 0 to 90080
Data columns (total 41 columns):
campaigns              0 non-null object
checksum               90081 non-null object
company_name           90081 non-null object
company_name_hash64    90081 non-null object
date_created_x         90081 non-null object
date_expired           90081 non-null object
date_updated           90081 non-null object
emails                 90081 non-null object
emails_src             90081 non-null object
html_desc_mode         90081 non-null object
id                     90081 non-null object
id_campaign            90081 non-null object
id_currency            90081 non-null object
id_language            90081 non-null object
id_project             90081 non-null object
id_region              90081 non-null object
id_salary_rate         90081 non-null object
id_similar_group       90081 non-null object
inactive               90081 non-null object
is_active              90081 non-null ob

In [22]:
full_df.to_csv("by_jobs_full.csv", sep='\t', header=True, index=False)

In [23]:
nessesary_df = full_df.reindex(columns=["id",
                                        "title",
                                        "lang_title",
                                        "title_normalized",
                                        "title_lemmas",
                                        "title_lemmas_tags",
                                        "title_tokens",                    
                                        "text", 
                                        "lang_text",
                                        "text_normalized",
                                        "text_lemmas",
                                        "text_lemmas_tags",
                                        "text_tokens",
                                        "url"])

In [24]:
print(nessesary_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90081 entries, 0 to 90080
Data columns (total 14 columns):
id                   90081 non-null object
title                90081 non-null object
lang_title           90081 non-null object
title_normalized     90081 non-null object
title_lemmas         0 non-null float64
title_lemmas_tags    0 non-null float64
title_tokens         0 non-null float64
text                 90081 non-null object
lang_text            90081 non-null object
text_normalized      0 non-null float64
text_lemmas          0 non-null float64
text_lemmas_tags     0 non-null float64
text_tokens          0 non-null float64
url                  90081 non-null object
dtypes: float64(7), object(7)
memory usage: 10.3+ MB
None


### Normalize text and title using text_preprocessing service

In [25]:
# drop prepared data
nessesary_df = nessesary_df.loc[10_000: ]

In [ ]:
%%time
for index, row in nessesary_df.iterrows():
    
    fields = ["title", "text"]
    for field in fields:
        text = row[field]
        r = requests.post(text_processing_url + config.STEM_TEXT_PATH,
                          json=text)
        nessesary_df.loc[index, field + "_normalized"] = r.text
        
        r = requests.post(text_processing_url + config.LEMM_TEXT_PATH,
                          json=text)
        nessesary_df.loc[index, field + "_lemmas"] = r.text
        
        r = requests.post(text_processing_url + config.TAG_TEXT_PATH,
                          json=text)
        nessesary_df.loc[index, field + "_lemmas_tags"] = r.text
        
        r = requests.post(text_processing_url + config.TOKEN_TEXT_PATH,
                          json=text)
        nessesary_df.loc[index, field + "_tokens"] = r.text
        
        r = requests.post(text_processing_url + config.DETECT_LANG_PATH ,
                          json=text)
        nessesary_df.loc[index, "lang_" + field] = r.text
        
    
    if index % 1000 == 0:
        file_name = "by_jobs_10K_to_" + str(index) + ".csv"
        print(index)
        nessesary_df.to_csv(file_name, sep='\t', header=True, index=None)
  

In [33]:
nessesary_df = nessesary_df[np.logical_or(nessesary_df["lang_text"]=="russian",
                                          nessesary_df["lang_text"]=="english")]

In [34]:
print(len(nessesary_df))
print(nessesary_df.info())

50809
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50809 entries, 10000 to 60808
Data columns (total 14 columns):
id                   50809 non-null object
title                50809 non-null object
lang_title           50809 non-null object
title_normalized     50809 non-null object
title_lemmas         50809 non-null object
title_lemmas_tags    50809 non-null object
title_tokens         50809 non-null object
text                 50809 non-null object
lang_text            50809 non-null object
text_normalized      50809 non-null object
text_lemmas          50809 non-null object
text_lemmas_tags     50809 non-null object
text_tokens          50809 non-null object
url                  50809 non-null object
dtypes: object(14)
memory usage: 5.8+ MB
None


In [35]:
nessesary_df.tail()

,id,title,lang_title,title_normalized,title_lemmas,title_lemmas_tags,title_tokens,text,lang_text,text_normalized,text_lemmas,text_lemmas_tags,text_tokens,url
60804,-6380895435018986256,Художник векторной графики. Отрисовать из jpg ...,russian,художник векторн график отрисова jpg a удален,художник векторный график рисовать jpg ai удал...,художник_NOUN векторный_ADJ график_NOUN рисова...,художник векторной графики отрисовать jpg ai у...,Необходимо отрисовать рисунок jpg в ai (2-3 шт),russian,необходим отрисова рисунок jpg a шт,необходимый рисовать рисунок jpg ai штука,необходимый_ADJ рисовать_VERB рисунок_NOUN jpg...,необходимо отрисовать рисунок jpg ai шт,https://www.fl.ru/projects/3107090/otrisovat-i...
60805,-5127522319541783233,Музыкальный руководитель,russian,музыкальн руководител,музыкальный руководитель,музыкальный_ADJ руководитель_NOUN,музыкальный руководитель,Режим работы:Полный рабочий деньЗарплата:320.0...,russian,реж работ полн рабоч музыкальн руководител спе...,режим работа полный рабочий музыкальный руково...,режим_NOUN работа_NOUN полный_ADJ рабочий_ADJ ...,режим работы полный рабочий музыкальный руково...,http://minsk.regiony.by/работа/вакансии/#!jobs...
60806,-2700786999414105615,Грузчик-комплектовщик,russian,,,,,"работа на складе или на производстве (в цеху),...",russian,работ склад производств цех машин комплектац п...,работа склад производство цех машина комплекта...,работа_NOUN склад_NOUN производство_NOUN цех_N...,работа складе производстве цеху машин комплект...,http://rdw.by/vakansii/gruzchik-komplektovshch...
60807,3725440889222302714,Создание бота для сообщества в социальной сети...,russian,создан бот сообществ социальн сет контакт удален,создание бовать сообщество социальный сеть кон...,создание_NOUN бовать_NOUN сообщество_NOUN соци...,создание бота сообщества социальной сети конта...,Я занимаюсь преподаванием и разработал некотор...,russian,занима преподаван разработа некотор количеств ...,заниматься преподавание разрабатывать некоторы...,заниматься_VERB преподавание_NOUN разрабатыват...,занимаюсь преподаванием разработал некоторое к...,https://www.fl.ru/projects/3106788/sozdanie-bo...
60808,3209976227911552970,Operations - Junior Specialist with Scandinavi...,english,oper junior specialist scandinavian languag,operation junior specialist scandinavian language,operation junior specialist scandinavian language,operations junior specialist scandinavian lang...,"Accenture is a global management consulting, t...",english,accentur global manag consult technolog servic...,accenture global management consulting technol...,accenture global management consulting technol...,accenture global management consulting technol...,https://www.careersinpoland.com/job/accenture/...


In [37]:
exist_df = pd.read_csv(os.path.join("headHunter_data", "by_jobs.csv"), sep='\t')
exist_df = exist_df.loc[np.logical_or(exist_df["lang_text"]=="russian",
                                      exist_df["lang_text"]=="english")]
exist_df.reset_index(drop=True, inplace=True)
print(exist_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 14 columns):
id                   10001 non-null int64
title                10001 non-null object
lang_title           10001 non-null object
title_normalized     9048 non-null object
title_lemmas         9048 non-null object
title_lemmas_tags    9048 non-null object
title_tokens         9048 non-null object
text                 10001 non-null object
lang_text            10001 non-null object
text_normalized      10000 non-null object
text_lemmas          10000 non-null object
text_lemmas_tags     10000 non-null object
text_tokens          10000 non-null object
url                  10001 non-null object
dtypes: int64(1), object(13)
memory usage: 1.1+ MB
None


In [38]:
nessesary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50809 entries, 10000 to 60808
Data columns (total 14 columns):
id                   50809 non-null object
title                50809 non-null object
lang_title           50809 non-null object
title_normalized     50809 non-null object
title_lemmas         50809 non-null object
title_lemmas_tags    50809 non-null object
title_tokens         50809 non-null object
text                 50809 non-null object
lang_text            50809 non-null object
text_normalized      50809 non-null object
text_lemmas          50809 non-null object
text_lemmas_tags     50809 non-null object
text_tokens          50809 non-null object
url                  50809 non-null object
dtypes: object(14)
memory usage: 5.8+ MB


In [40]:
exist_df = exist_df.append(nessesary_df, ignore_index=True, sort=True)
print("Final size of dataset =", len(exist_df))

Final size of dataset = 60810


In [42]:
exist_df.to_csv("by_jobs.csv", sep='\t', header=True, index=None)

### Check saved data

In [21]:
exist_df

,id,lang_text,lang_title,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url,requirement_normalized
0,6470673679709264762,russian,russian,Образование:Общее среднееХарактер работы:Посто...,образование общий среднеехарактирать работа эл...,образование_NOUN общий_ADJ среднеехарактирать_...,образован общ среднеехарактер работ электрогаз...,образование общее среднеехарактер работы элект...,Электрогазосварщик,электрогазосварщик,электрогазосварщик_NOUN,электрогазосварщик,электрогазосварщик,http://minsk.regiony.by/работа/вакансии/#!jobs...,NaN
1,299281100178254203,russian,russian,Сделать не сложную по дизайну комнату произвол...,делать сложный дизайн комната произвольный раз...,делать_VERB сложный_ADJ дизайн_NOUN комната_NO...,сдела сложн дизайн комнат произвольн размер ви...,сделать сложную дизайну комнату произвольных р...,Проектирование. Комната в Архикад (удаленно),проектирование комната архикад удаленно,проектирование_NOUN комната_NOUN архикад_NOUN ...,проектирован комнат архикад удален,проектирование комната архикад удаленно,https://www.fl.ru/projects/3103044/komnata-v-a...,NaN
2,-4517560807677477007,russian,russian,Всем привет! Мы ищем крутую компетентную студи...,весь привет искать крутой компетентной студия ...,весь_DET привет_NOUN искать_VERB крутой_ADJ ко...,всем привет ищ крут компетентн студ разработк ...,всем привет ищем крутую компетентную студию ра...,Программист для сотовых телефонов и КПК. Ищем ...,программист сотовый телефон кпкать искать студ...,программист_NOUN сотовый_ADJ телефон_NOUN кпка...,программист сотов телефон кпк ищ студ разработ...,программист сотовых телефонов кпк ищем студию ...,https://www.fl.ru/projects/3102743/ischem-stud...,NaN
3,8871098156798776528,russian,russian,"Необходимо убрать шумы все и мелодию, чтобы ос...",необходимый убирать шум мелодия оставаться чис...,необходимый_ADJ убирать_VERB шум_NOUN мелодия_...,необходим убра шум мелод оста чист голос ссылк,необходимо убрать шумы мелодию остался чистый ...,Аудиомонтажер. Подавление шума на аудиозаписи ...,аудиомонтажирать подавление шум аудиозапись уд...,аудиомонтажирать_NOUN подавление_NOUN шум_NOUN...,аудиомонтажер подавлен шум аудиозапис удален,аудиомонтажер подавление шума аудиозаписи удал...,https://www.fl.ru/projects/3102912/podavlenie-...,NaN
4,6075012463033739773,russian,russian,Режим работы:Сменный график работыЗарплата:600...,режим работа сменный график разрядавакансия сы...,режим_NOUN работа_NOUN сменный_ADJ график_NOUN...,реж работ смен график разрядаваканс сыродел со...,режим работы сменный график разрядавакансия сы...,Сыродел по созреванию сыров,сыродеть созревание сыр,сыродеть_VERB созревание_NOUN сыр_NOUN,сыродел созреван сыр,сыродел созреванию сыров,http://hrodna.regiony.by/работа/вакансии/#!job...,NaN
5,-5691828309504281472,russian,russian,Предприятию на постоянную работу требуются шве...,предприятие постоянный работа требоваться швея...,предприятие_NOUN постоянный_ADJ работа_NOUN тр...,предприят постоя работ треб шве обязан пош жен...,предприятию постоянную работу требуются швеи о...,Швея-портной,NaN,NaN,NaN,NaN,https://brest.jobs.tut.by/vacancy/19424976,обязан пош женск детск одежд различн требован ...
6,9030264139348671477,russian,russian,Образование:Общее среднееХарактер работы:Посто...,образование общий среднеехарактирать работа ра...,образование_NOUN общий_ADJ среднеехарактирать_...,образован общ среднеехарактер работ разрядавак...,образование общее среднеехарактер работы разря...,Монтажник санитарно-технических систем и обору...,монтажник система оборудование,монтажник_NOUN система_NOUN оборудование_NOUN,монтажник сист оборудован,монтажник систем оборудования,http://minsk.regiony.by/работа/вакансии/#!jobs...,NaN
7,-8145737579285623957,russian,russian,"В магазин ""Птушка"" по ул.Я.Коласа требуется уб...",магазин птушка требоваться уборщица ставка обя...,магазин_NOUN птушка_NOUN требоваться_VERB убор...,магазин птушк треб уборщиц ставк обязан

In [23]:
exist_df = pd.read_csv("labeled_here_1_10k.csv", sep='\t',)
exist_df.info()   
exist_df2 = pd.read_csv("labeled_here_2_10k.csv", sep='\t',)
exist_df2.info()
exist_df3 = pd.read_csv("labeled_here_3_10k.csv", sep='\t',)
exist_df3.info()
exist_df4 = pd.read_csv("labeled_here_4_10k.csv", sep='\t',)
exist_df4.info()
exist_df5 = pd.read_csv("labeled_here_5_10k.csv", sep='\t',)
exist_df5.info()
exist_df6 = pd.read_csv("labeled_here_6_10k.csv", sep='\t',)
exist_df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 15 columns):
id                        10001 non-null int64
lang_text                 10001 non-null object
lang_title                10001 non-null object
text                      10001 non-null object
text_lemmas               10000 non-null object
text_lemmas_tags          10000 non-null object
text_normalized           10000 non-null object
text_tokens               10000 non-null object
title                     10001 non-null object
title_lemmas              9048 non-null object
title_lemmas_tags         9048 non-null object
title_normalized          9048 non-null object
title_tokens              9048 non-null object
url                       10001 non-null object
requirement_normalized    4575 non-null object
dtypes: int64(1), object(14)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 15 columns):
id               

In [25]:
labeled_all = pd.concat([exist_df, exist_df2, exist_df3, exist_df4, exist_df5, exist_df6], ignore_index=True)

In [26]:
labeled_all.info()
labeled_all

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60815 entries, 0 to 60814
Data columns (total 15 columns):
id                        60815 non-null int64
lang_text                 60815 non-null object
lang_title                60815 non-null object
text                      60815 non-null object
text_lemmas               60804 non-null object
text_lemmas_tags          60804 non-null object
text_normalized           60804 non-null object
text_tokens               60804 non-null object
title                     60815 non-null object
title_lemmas              55548 non-null object
title_lemmas_tags         55548 non-null object
title_normalized          55551 non-null object
title_tokens              55551 non-null object
url                       60815 non-null object
requirement_normalized    20721 non-null object
dtypes: int64(1), object(14)
memory usage: 7.0+ MB


,id,lang_text,lang_title,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url,requirement_normalized
0,8253926506452218683,russian,russian,\r\r\r\n Предлагаем большой ассортимент растен...,предлагать большой ассортимент растение собств...,предлагать_VERB большой_ADJ ассортимент_NOUN р...,предлага больш ассортимент растен собствен про...,предлагаем большой ассортимент растений собств...,Саженцы декоративных и плодовых культур. Могил...,саженец декоративный плодовый культура могилев...,саженец_NOUN декоративный_ADJ плодовый_ADJ кул...,саженц декоративн плодов культур могил гарант,саженцы декоративных плодовых культур могилев ...,http://slanet.by/Sazhency_dekorativnyh_i_plodo...,NaN
1,-6951509096940958272,russian,russian,Срочно отрисовать не сложный ч/б логотип в век...,срочно рисовать сложный логотип вектор,срочно_ADV рисовать_VERB сложный_ADJ логотип_N...,срочн отрисова сложн логотип вектор,срочно отрисовать сложный логотип вектор,Дизайнер логотипов. Срочно отрисовать не сложн...,дизайнер логотип срочно рисовать сложный логот...,дизайнер_NOUN логотип_NOUN срочно_ADV рисовать...,дизайнер логотип срочн отрисова сложн логотип ...,дизайнер логотипов срочно отрисовать сложный л...,https://www.fl.ru/projects/3085095/srochno-otr...,NaN
2,8709047890904426760,russian,russian,курсовая работа по Экономика фирмы \r\r\nдисци...,курсовой работа экономика фирма дисциплина эко...,курсовой_ADJ работа_NOUN экономика_NOUN фирма_...,курсов работ экономик фирм дисциплин экономик ...,курсовая работа экономика фирмы дисциплина эко...,#СРОЧНО!курсовая работа по Экономика фирмы (уд...,срочно курсовой работа экономика фирма удаленно,срочно_ADV курсовой_ADJ работа_NOUN экономика_...,срочн курсов работ экономик фирм удален,срочно курсовая работа экономика фирмы удаленно,https://www.fl.ru/projects/3085048/srochnokurs...,NaN
3,3844137534604474003,russian,russian,Воспользуюсь услугами помощника по хозяйству в...,воспользоваться услуга помощник хозяйство част...,воспользоваться_VERB услуга_NOUN помощник_NOUN...,воспольз услуг помощник хозяйств частн дом пре...,воспользуюсь услугами помощника хозяйству част...,Помощник по хозяйству.,помощник хозяйство,помощник_NOUN хозяйство_NOUN,помощник хозяйств,помощник хозяйству,https://www.kufar.by/Брест/Вакансии/Помощник_п...,NaN
4,4924993067476806508,russian,russian,"И показать как делать, так как сайтов много.\r...",показывать делать сайт сайт vds скрипть,показывать_VERB делать_VERB сайт_NOUN сайт_NOU...,показа дела сайт сайт vds скрипт,показать делать сайтов сайт vds скрипт,Перенести сайт с на (удаленно),переносить сайт удаленно,переносить_VERB сайт_NOUN удаленно_ADV,перенест сайт удален,перенести сайт удаленно,https://www.fl.ru/projects/3085032/perenesti-s...,NaN
5,-8546375017518166188,russian,russian,"Добрый вечер коллеги, дорогие фрилансеры.\r\r\...",добрый вечер коллега дорогой фрилансер сердечн...,добрый_ADJ вечер_NOUN коллега_NOUN дорогой_ADJ...,добр вечер коллег дорог фрилансер сердечн позд...,добрый вечер коллеги дорогие фрилансеры сердеч...,С наступающим Новым Годом! (удаленно),наступать новый год удаленно,наступать_VERB новый_ADJ год_NOUN удаленно_ADV,наступа нов год удален,наступающим новым годом удаленно,https://www.fl.ru/projects/3085092/s-nastupayu...,NaN
6,-4677914238886049045,russian,russian,Только 31 декабря!\r\r\nВсего за 777 рублей!\r...,декабрь рубль контактный данные адрес телефон ...,декабрь_NOUN рубль_NOUN контактный_ADJ данные_...,декабр рубл контактн дан адрес телефон сайт em...,декабря рублей контактные данные адрес телефон...,База данных - Организации РФ! (удаленно),база данные организация рф удаленно,база_NOUN данные_NOUN организация_NOUN рф_PROP...,баз дан организац рф удален,база данных организации рф удаленно,https://www.fl.ru/projects/3085030/baza-dannyi...,NaN
7,1454265527494451396,russian,russian,"Крупнейший образовательный центр ""Лидер"" пригл...",крупный образовательный центр лидер приглашать...,крупный

In [27]:
labeled_all.to_csv("labeled_jooble_60K.csv", sep='\t', header=True, index=False)

In [28]:
exist_df = pd.read_csv("labeled_jooble_60K.csv", sep='\t',)
exist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60815 entries, 0 to 60814
Data columns (total 15 columns):
id                        60815 non-null int64
lang_text                 60815 non-null object
lang_title                60815 non-null object
text                      60815 non-null object
text_lemmas               60804 non-null object
text_lemmas_tags          60804 non-null object
text_normalized           60804 non-null object
text_tokens               60804 non-null object
title                     60815 non-null object
title_lemmas              55548 non-null object
title_lemmas_tags         55548 non-null object
title_normalized          55551 non-null object
title_tokens              55551 non-null object
url                       60815 non-null object
requirement_normalized    20721 non-null object
dtypes: int64(1), object(14)
memory usage: 7.0+ MB
